# Analyse touch chart model after training

In this notebook, I analyse the touch chart model after training. The goal is to visualise the predicted mesh and ground truth mesh given a collected tactile image and corresponding point cloud.

In [62]:
import numpy as np
import torch
from data_making.dataset_touch import TouchChartDataset
import os
import results
from results import runs_touch
from model import model_touch
from utils import utils_mesh
import data
import trimesh
import matplotlib.pyplot as plt
%matplotlib qt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Import dataset:

In [47]:
touch_chart_path = os.path.join(os.path.dirname(results.__file__), 'touch_charts_gt.npy')
dataset = TouchChartDataset(touch_chart_path)

Load default touch chart needed for prediction:

In [54]:
chart_location = os.path.join(os.path.dirname(data.__file__), 'touch_chart.obj')
initial_verts, initial_faces = utils_mesh.load_mesh_touch(chart_location)
initial_verts = torch.unsqueeze(initial_verts, 0)

Load model and weights:

In [64]:
runs_touch_name = '14_12_1041'

In [65]:
modello = model_touch.Encoder()

# load weights
weights_path = os.path.join(os.path.dirname(runs_touch.__file__), runs_touch_name, 'weights.pt')
modello.load_state_dict(torch.load(weights_path, map_location=torch.device(device)))
modello.eval()

Encoder(
  (CNN_layers): Sequential(
    (0): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      )
      (activation): Sequential(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
      )
    )
    (1): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(32, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(

Now, I sample a random tactile image and corresponding point cloud. 

In [67]:
val_sample_index = np.random.choice(len(dataset))

with torch.no_grad():
    # Load x and y
    tactile_img, pointcloud_gt = dataset[val_sample_index][0], dataset[val_sample_index][1]

    # Adjust dimensions of x and y
    tactile_img = torch.unsqueeze(tactile_img, 0)

    # predict verts
    predicted_verts = modello(tactile_img, initial_verts)[0]
    
    # predicted mesh
    predicted_mesh = trimesh.Trimesh(predicted_verts, initial_faces)
    predicted_pointcloud = utils_mesh.mesh_to_pointcloud(predicted_mesh, 500)
    
    # plot point cloud gt and predicted point cloud
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    x1 = predicted_pointcloud[:, 0]
    y1 = predicted_pointcloud[:, 1]
    z1 = predicted_pointcloud[:, 2]
    
    x2 = pointcloud_gt[:, 0]
    y2 = pointcloud_gt[:, 1]
    z2 = pointcloud_gt[:, 2] 
    
    ax.scatter(x1, y1, z1)
    ax.scatter(x2, y2, z2)
    
    plt.show()